In [1]:
import pandas as pd
import numpy as np
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

from gensim import corpora, models, similarities


C:\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",explanation edits made username hardcore metal...,<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",aww match background colour seemingly stuck th...,<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",hey man really trying edit war guy constantly ...,<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",make real suggestion improvement wondered sect...,<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",sir hero chance remember page,<class 'dict'>
5,5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"['congratulation', 'well', 'use', 'tool', 'wel...",congratulation well use tool well talk,<class 'dict'>
6,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"['cocksucker', 'piss', 'around', 'work']",cocksucker piss around work,<class 'dict'>
7,7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,"['vandalism', 'matt', 'shirvington', 'article'...",vandalism matt shirvington article reverted pl...,<class 'dict'>
8,8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"['sorry', 'word', 'nonsense', 'offensive', 'an...",sorry word nonsense offensive anyway intending...,<class 'dict'>
9,9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,"['alignment', 'subject', 'contrary', 'dulithgow']",alignment subject contrary dulithgow,<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

20.686476230621338


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...,"{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...",aww match background colour seemingly stuck th...,"{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...,"{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...",make real suggestion improvement wondered sect...,"{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]",sir hero chance remember page,"{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
# MULTICLASS PREDICTION

In [6]:
classLabel = {
    0: "neutral",
    1: "toxic",
    2 : "severe_toxic",
    3 : "obscene",
    4 : "threat",
    5 : "insult",
    6 : "identity_hate" 
}


In [7]:
y = np.empty((len(train['cleanWordsAsText']),),dtype=object)
allTextToxicTrain = dict()
for idx in classLabel:
    if classLabel[idx] != "neutral":
        T = np.where(train[classLabel[idx]] == 1)[0]
        allTextToxicTrain[idx] = T
        for i in T:
            if y[i] is None:
                y[i] = [idx]                
            else:
                y[i].append(idx)
indxsOfNeutralTexts = np.where(y == None) 
y[indxsOfNeutralTexts] = [[0]]
indxsOfNeutralTexts = indxsOfNeutralTexts[0]

allTextsNoToxicTrain = [str(train['cleanWordsAsText'][x]) for x in indxsOfNeutralTexts]

idxList = []
for i in allTextToxicTrain.keys():
    #allTextToxicTrain[i] = [str(train['cleanWordsAsText'][j]) for j in allTextToxicTrain[i]]
    idxList = np.unique(np.append(idxList, allTextToxicTrain[i]))
allTextToxicTrain = [str(train['cleanWordsAsText'][j]) for j in idxList]

In [8]:
columns = ["idExp","numFeatures", "algorithm", "Nfolds", "accuaracy", "logloss", "fmeasure"]
dfTestResults = pd.DataFrame(columns=columns)


In [9]:
modelWord2VecTrain = models.Word2Vec(train['listOfCleanWords'].tolist(), min_count=1)
#say_vector = modelWord2VecTrain['explanation']  # get vector for word
#say_vector

In [10]:
vectorizedTrainW2V = np.empty(len(train['listOfCleanWords']), dtype=list)
for i, lstOfCleanWords in enumerate(train['listOfCleanWords']): 
    size = len(lstOfCleanWords)
    if size > 0:
        vector = np.zeros(len(modelWord2VecTrain[lstOfCleanWords[0]]))
        for x in range(size):
            vector += modelWord2VecTrain[lstOfCleanWords[x]]
        vectorizedTrainW2V[i] = vector / size

C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [11]:
vectorizedTrainW2V

array([ array([ -3.00136152e-01,  -5.71729056e-01,  -4.96090920e-01,
        -1.17840051e-01,  -6.81519689e-01,   1.59711916e-02,
         4.99034632e-02,  -5.65436888e-01,  -3.31596926e-01,
        -2.43633995e-01,  -1.11024374e-01,  -2.47723006e-01,
        -2.08194790e-01,  -4.69958825e-02,  -1.05695644e-02,
        -3.08239269e-02,  -4.90374701e-04,  -4.76884366e-01,
         7.65669723e-01,  -3.59594397e-01,  -1.15618177e-01,
         3.12664044e-01,   4.33685690e-01,   2.18956311e-01,
        -1.04938791e-02,  -1.54937528e-01,  -2.14078129e-01,
        -4.98786159e-01,   9.53592377e-01,   1.25305617e+00,
        -5.68193221e-01,   8.00695183e-02,   8.92788202e-01,
         7.43446916e-02,  -4.74748916e-01,   1.17665295e+00,
        -3.20254573e-01,   5.71895715e-01,  -2.82631979e-01,
        -2.29279274e-01,  -1.21729387e+00,  -4.88144905e-01,
        -9.32447399e-01,  -4.94489278e-01,   6.45974980e-01,
         5.75190730e-01,   1.51236555e-01,  -1.01207273e+00,
         4.43065